In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [7]:
# 1.获取数据集
# 2.基本数据处理
# 2.1 缩小数据范围
# 2.2 选择时间特征
# 2.3 去掉签到较少的地方
# 2.4 确定特征值和目标值
# 2.5 分割数据集
# 3.特征工程 -- 特征预处理(标准化)
# 4.机器学习 -- knn+cv
# 5.模型评估

In [2]:
# 1.获取数据集
data = pd.read_csv("./data/FBlocation/train.csv")

In [3]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949


In [4]:
data.describe()

,row_id,x,y,accuracy,time,place_id
count,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07,2.911802e+07
mean,1.455901e+07,4.999770e+00,5.001814e+00,8.284912e+01,4.170104e+05,5.493787e+09
std,8.405649e+06,2.857601e+00,2.887505e+00,1.147518e+02,2.311761e+05,2.611088e+09
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000016e+09
25%,7.279505e+06,2.534700e+00,2.496700e+00,2.700000e+01,2.030570e+05,3.222911e+09
50%,1.455901e+07,5.009100e+00,4.988300e+00,6.200000e+01,4.339220e+05,5.518573e+09
75%,2.183852e+07,7.461400e+00,7.510300e+00,7.500000e+01,6.204910e+05,7.764307e+09
max,2.911802e+07,1.000000e+01,1.000000e+01,1.033000e+03,7.862390e+05,9.999932e+09


In [5]:
data.shape

(29118021, 6)

In [9]:
# 2.基本数据处理
# 2.1 缩小数据范围
partial_data = data.query("x>2.0 & x<2.5 & y>2.0 & y<2.5")

In [10]:
partial_data.head()

,row_id,x,y,accuracy,time,place_id
163,163,2.1663,2.3755,84,669737,3869813743
310,310,2.3695,2.2034,3,234719,2636621520
658,658,2.3236,2.1768,66,502343,7877745055
1368,1368,2.2613,2.3392,73,319822,9775192577
1627,1627,2.3331,2.0011,66,595084,6731326909


In [11]:
partial_data.shape

(71664, 6)

In [13]:
# 2.2 选择时间特征
partial_data["time"].head()

163     669737
310     234719
658     502343
1368    319822
1627    595084
Name: time, dtype: int64

In [16]:
time = pd.to_datetime(partial_data["time"], unit="s")
# 脱敏
time.head()

163    1970-01-08 18:02:17
310    1970-01-03 17:11:59
658    1970-01-06 19:32:23
1368   1970-01-04 16:50:22
1627   1970-01-07 21:18:04
Name: time, dtype: datetime64[ns]

In [18]:
time = pd.DatetimeIndex(time)
time

DatetimeIndex(['1970-01-08 18:02:17', '1970-01-03 17:11:59',
               '1970-01-06 19:32:23', '1970-01-04 16:50:22',
               '1970-01-07 21:18:04', '1970-01-02 03:14:59',
               '1970-01-07 03:45:16', '1970-01-05 03:28:43',
               '1970-01-01 18:59:03', '1970-01-09 07:50:12',
               ...
               '1970-01-09 20:03:34', '1970-01-08 09:26:50',
               '1970-01-07 04:45:59', '1970-01-07 22:36:18',
               '1970-01-06 23:29:43', '1970-01-03 12:31:26',
               '1970-01-04 15:19:20', '1970-01-01 20:49:14',
               '1970-01-03 09:17:37', '1970-01-02 20:34:43'],
              dtype='datetime64[ns]', name='time', length=71664, freq=None)

In [19]:
time.hour

Int64Index([18, 17, 19, 16, 21,  3,  3,  3, 18,  7,
            ...
            20,  9,  4, 22, 23, 12, 15, 20,  9, 20],
           dtype='int64', name='time', length=71664)

In [20]:
time.day

Int64Index([8, 3, 6, 4, 7, 2, 7, 5, 1, 9,
            ...
            9, 8, 7, 7, 6, 3, 4, 1, 3, 2],
           dtype='int64', name='time', length=71664)

In [21]:
partial_data["hour"] = time.hour
partial_data["day"] = time.day
partial_data["weekday"] = time.weekday

/Users/sherwin/workspaces/ai/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sherwin/workspaces/ai/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sherwin/workspaces/ai/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

In [22]:
partial_data.head()

,row_id,x,y,accuracy,time,place_id,hour,day,weekday
163,163,2.1663,2.3755,84,669737,3869813743,18,8,3
310,310,2.3695,2.2034,3,234719,2636621520,17,3,5
658,658,2.3236,2.1768,66,502343,7877745055,19,6,1
1368,1368,2.2613,2.3392,73,319822,9775192577,16,4,6
1627,1627,2.3331,2.0011,66,595084,6731326909,21,7,2


In [23]:
# 2.3 去掉签到较少的地方
partial_data.head()

,row_id,x,y,accuracy,time,place_id,hour,day,weekday
163,163,2.1663,2.3755,84,669737,3869813743,18,8,3
310,310,2.3695,2.2034,3,234719,2636621520,17,3,5
658,658,2.3236,2.1768,66,502343,7877745055,19,6,1
1368,1368,2.2613,2.3392,73,319822,9775192577,16,4,6
1627,1627,2.3331,2.0011,66,595084,6731326909,21,7,2


In [31]:
place_count = partial_data.groupby("place_id").count()

In [32]:
place_count.head()

,row_id,x,y,accuracy,time,hour,day,weekday
place_id,,,,,,,,
1006234733,1,1,1,1,1,1,1,1
1008823061,4,4,4,4,4,4,4,4
1012580558,3,3,3,3,3,3,3,3
1025585791,21,21,21,21,21,21,21,21
1026507711,220,220,220,220,220,220,220,220


In [36]:
place_count = place_count[place_count["row_id"]>3]
place_count.head()

,row_id,x,y,accuracy,time,hour,day,weekday
place_id,,,,,,,,
1008823061,4,4,4,4,4,4,4,4
1025585791,21,21,21,21,21,21,21,21
1026507711,220,220,220,220,220,220,220,220
1032417180,10,10,10,10,10,10,10,10
1040557418,123,123,123,123,123,123,123,123


In [40]:
partial_data = partial_data[partial_data["place_id"].isin(place_count.index)]

In [41]:
partial_data.shape

(69264, 9)

In [42]:
partial_data.head()

,row_id,x,y,accuracy,time,place_id,hour,day,weekday
163,163,2.1663,2.3755,84,669737,3869813743,18,8,3
310,310,2.3695,2.2034,3,234719,2636621520,17,3,5
658,658,2.3236,2.1768,66,502343,7877745055,19,6,1
1368,1368,2.2613,2.3392,73,319822,9775192577,16,4,6
1627,1627,2.3331,2.0011,66,595084,6731326909,21,7,2


In [43]:
# 2.4 确定特征值和目标值
x = partial_data[["x", "y", "accuracy", "hour", "day", "weekday"]]
y = partial_data["place_id"]

In [44]:
x.head()

,x,y,accuracy,hour,day,weekday
163,2.1663,2.3755,84,18,8,3
310,2.3695,2.2034,3,17,3,5
658,2.3236,2.1768,66,19,6,1
1368,2.2613,2.3392,73,16,4,6
1627,2.3331,2.0011,66,21,7,2


In [45]:
y.head()

163     3869813743
310     2636621520
658     7877745055
1368    9775192577
1627    6731326909
Name: place_id, dtype: int64

In [46]:
# 2.5 分割数据集
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2, test_size=0.25)

In [47]:
x_train.head()

,x,y,accuracy,hour,day,weekday
19509166,2.3217,2.2029,1,0,8,3
20577315,2.4800,2.2129,175,20,7,2
24044078,2.2389,2.3447,79,3,8,3
11279021,2.0850,2.2789,119,1,9,4
19491154,2.2408,2.0092,168,0,8,3


In [48]:
# 3.特征工程 -- 特征预处理(标准化)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.fit_transform(x_test)

In [57]:
# 4.机器学习 -- knn+cv
# 4.1 实例化一个训练器
estimator = KNeighborsClassifier()

# 4.2 交叉验证,网格搜索实现
param_grid = {"n_neighbors": [3, 5, 7, 9]}
estimator = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=10, n_jobs=4)

# 4.3 模型训练
estimator.fit(x_train, y_train)

/Users/sherwin/workspaces/ai/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'n_neighbors': [3, 5, 7, 9]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [62]:
# 5.模型评估
# 5.1 准确率输出
score_ret = estimator.score(x_test, y_test)
print("准确率为:\n", score_ret)

准确率为:
 0.3697158697158697


In [63]:
# 5.2 预测结果
y_pre = estimator.predict(x_test)
print("预测值是:\n", y_pre)

预测值是:
 [3105513359 8980163153 1247398579 ... 1891783132 8169595806 3661555534]


In [64]:
# 5.3 其他结果输出
print("最好的模型是:\n", estimator.best_estimator_)
print("最好的结果是:\n", estimator.best_score_)
print("所有的结果是:\n", estimator.cv_results_)

最好的模型是:
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
最好的结果是:
 0.3621506121506122
所有的结果是:
 {'mean_fit_time': array([0.07974284, 0.06533892, 0.06671908, 0.06837542]), 'std_fit_time': array([0.00468629, 0.00599286, 0.00193125, 0.00476899]), 'mean_score_time': array([0.28204682, 0.27073274, 0.30490429, 0.31999531]), 'std_score_time': array([0.01271002, 0.03878049, 0.03855714, 0.06601039]), 'param_n_neighbors': masked_array(data=[3, 5, 7, 9],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([0.32790657, 0.34330207, 0.34383295, 0.33781632]), 'split1_test_score': array([0.32915473, 0.34294413, 0.34043696, 0.3357808 ]), 'split2_test_score': array([0.3426484 , 0.3483105 , 0.34703196, 0.34191781]), 'split3_test_s